In [1]:
# -*- coding: utf-8 -*-
import numpy as np
import pandas as pd
import os

from sklearn.model_selection import train_test_split
from sklearn.model_selection import KFold, StratifiedKFold, GroupKFold
from sklearn.metrics import accuracy_score,auc,roc_curve
from sklearn.preprocessing import LabelBinarizer,StandardScaler

from sklearn import model_selection as cv
from sklearn.model_selection import cross_val_score

from sklearn import svm
from sklearn import tree
from sklearn.neural_network import MLPClassifier
from sklearn.ensemble import RandomForestRegressor
from sklearn.ensemble import AdaBoostClassifier

import matplotlib as mpl
import matplotlib.pyplot as plt

In [2]:
from dbn.tensorflow import SupervisedDBNClassification
from sklearn.ensemble import RandomForestClassifier

Instructions for updating:
non-resource variables are not supported in the long term


In [3]:
from tensorflow.keras.layers import Input,Dense,LSTM,GRU,BatchNormalization
from tensorflow.keras.layers import PReLU
from tensorflow.keras.models import Model
from tensorflow.keras.optimizers import Adam
import tensorflow.keras as K
import tensorflow as tf
from sklearn.metrics import mean_absolute_error as MAE
import matplotlib.pyplot as plt
import matplotlib.ticker as ticker

In [4]:
df = pd.read_csv('./dataset/2.storke_combine_code_multi_balance.csv')
df

,NIHSS,original_shape_Elongation,original_firstorder_Skewness,original_glcm_SumSquares,original_gldm_SmallDependenceLowGrayLevelEmphasis,wavelet-LLH_glrlm_GrayLevelVariance,wavelet-LLH_glrlm_ShortRunEmphasis,wavelet-LLH_gldm_GrayLevelVariance,wavelet-LLH_glszm_GrayLevelNonUniformity,wavelet-LHL_firstorder_90Percentile,...,Drinkalcohol _1.0,Drinkalcohol _2.0,Drinkalcohol _nan,Drinkalcohol _-1,OCSP_4.0,OCSP_3.0,OCSP_2.0,OCSP_1.0,OCSP_nan,OCSP_-1
0,0,0.565605,0.470144,0.373676,0.029411,0.000000,0.431860,0.000000,0.160000,0.491783,...,1,0,0,0,0,1,0,0,0,0
1,0,0.451330,0.392714,0.068449,0.106469,0.000000,0.495983,0.000000,0.160000,0.497301,...,1,0,0,0,1,0,0,0,0,0
2,0,0.612372,0.851287,0.069507,0.273280,0.000000,0.863248,0.000000,0.160000,0.178226,...,1,0,0,0,1,0,0,0,0,0
3,0,0.489943,0.405634,0.313635,0.066088,0.000000,0.457018,0.000000,0.160000,0.372396,...,1,0,0,0,1,0,0,0,0,0
4,0,0.653883,0.291065,0.236634,0.056482,0.000000,0.203632,0.000000,0.160000,0.326073,...,1,0,0,0,0,0,1,0,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
621,2,0.126109,0.447496,0.036365,0.934492,0.123367,0.062207,0.929336,0.104301,0.069522,...,1,0,0,0,0,0,1,0,0,0
622,2,0.458162,0.712629,0.361885,0.134446,0.498550,0.636870,0.061614,0.162788,0.234031,...,1,0,0,0,0,1,0,0,0,0
623,2,0.580937,0.412865,0.122301,0.643417,0.000000,0.273111,0.000198,0.488470,0.266404,...,1,0,0,0,0,0,1,0,0,0
624,2,0.311074,0.637602,0.023883,0.302411,0.000000,0.560780,0.223492,0.596927,0.070366,...,0,0,0,0,0,0,1,0,0,0


In [5]:
Y = np.array(df.pop('NIHSS'))
X = np.array(df)

In [6]:
x_train, x_test, y_train, y_test = train_test_split(X, Y, test_size=0.3, stratify=Y,random_state = 3)

In [7]:
x_train.shape

(438, 90)

In [8]:
Class = [0,1,2]
Class_dict = dict(zip(Class, range(len(Class))))
Class_dict
lb = LabelBinarizer()
lb.fit(list(Class_dict.values()))
y_train_labels = lb.transform(y_train)

In [9]:
x_train_lstm = x_train.reshape(x_train.shape[0],1,x_train.shape[1])
x_test_lstm = x_test.reshape(x_test.shape[0],1,x_test.shape[1])

In [10]:
def Predict(X):
    RNN_test_label = []
    Class = [0,1,2]
    Class_dict = dict(zip(Class, range(len(Class))))
    Class_dict
    for i in range(0,X.shape[0]):
        RNN_test_label.append(Class_dict[np.argmax(X[i])])
    RNN_test_label = np.array(RNN_test_label,dtype = 'int64')
    return RNN_test_label

In [11]:
#DNN
def buildDNN(layer1,layer2,n_class):
    init = K.initializers.glorot_uniform(seed=1)
    simple_adam = tf.keras.optimizers.Adam()
    model = K.models.Sequential()
    model.add(K.layers.Dense(units=layer1, input_dim=90, kernel_initializer=init, activation='relu'))
    model.add(K.layers.Dense(units=layer2, kernel_initializer=init, activation='relu'))
    model.add(K.layers.Dense(units=n_class, kernel_initializer=init, activation='softmax'))
    model.compile(loss='categorical_crossentropy', optimizer=simple_adam, metrics=['accuracy'])
    model.summary()
    return model

Model: "sequential"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 dense (Dense)               (None, 80)                7280      
                                                                 
 dense_1 (Dense)             (None, 80)                6480      
                                                                 
 dense_2 (Dense)             (None, 3)                 243       
                                                                 
Total params: 14,003
Trainable params: 14,003
Non-trainable params: 0
_________________________________________________________________


In [12]:
#DBN
def buildDBN(layer1,layer2,epoca,K=500):
    
    # cantidad de neuronas ocultas 
    hidden_layers = []
    hidden_layers.append( int(layer1))
    hidden_layers.append( int(layer2))

    DBN_classifier = SupervisedDBNClassification(hidden_layers_structure = hidden_layers,
                                                    learning_rate_rbm=0.05,
                                                    learning_rate=0.1,
                                                    n_epochs_rbm=epoca,
                                                    n_iter_backprop=K,
                                                    batch_size=32,
                                                    activation_function='relu',
                                                    dropout_p=0.2)
    return DBN_classifier

dbn = buildDBN(layer1=64,layer2=64,epoca=20,K=1000)

In [13]:
#LSTM_RNN
def buildLSTM(timeStep,inputColNum,outStep,learnRate=1e-4):

    inputLayer = Input(shape=(timeStep,inputColNum))

    middle = LSTM(100,activation='tanh')(inputLayer)
    middle = Dense(100,activation='tanh')(middle)

    outputLayer = Dense(outStep)(middle)
    
    model = Model(inputs=inputLayer,outputs=outputLayer)
    optimizer = Adam(learning_rate=learnRate)
    model.compile(optimizer=optimizer,loss='mse') 
    model.summary()
    return model

lstm = buildLSTM(timeStep=1,inputColNum=90,outStep=3,learnRate=1e-4)

Model: "model"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 input_1 (InputLayer)        [(None, 1, 90)]           0         
                                                                 
 lstm (LSTM)                 (None, 100)               76400     
                                                                 
 dense_3 (Dense)             (None, 100)               10100     
                                                                 
 dense_4 (Dense)             (None, 3)                 303       
                                                                 
Total params: 86,803
Trainable params: 86,803
Non-trainable params: 0
_________________________________________________________________


In [14]:
_N_FOLDS = 4 
kf = StratifiedKFold(n_splits=4, random_state=2020, shuffle=True)  
_N_CLASS = 3

In [15]:
oof_train1 = np.zeros((x_train.shape[0], _N_CLASS))  
oof_test1 = np.empty((x_test.shape[0], _N_CLASS))  

for i, (train_index, test_index) in enumerate(kf.split(x_train,y_train)):
    kf_X_train = x_train[train_index]
    kf_y_train = y_train[train_index]  
    kf_y_train = lb.transform(kf_y_train)
    kf_X_test = x_train[test_index] 

    dnn.fit(kf_X_train, kf_y_train,batch_size=20, epochs=100, shuffle=True, verbose=1)  
    
    oof_train1[test_index] = dnn.predict(kf_X_test)  
    oof_test1 += dnn.predict(x_test)  

oof_test1 /= _N_FOLDS 

Train on 328 samples
Epoch 1/100
328/328 [==============================] - 0s 194us/sample - loss: 0.9799 - acc: 0.5549
Epoch 2/100
328/328 [==============================] - 0s 27us/sample - loss: 0.7947 - acc: 0.7165
Epoch 3/100
328/328 [==============================] - 0s 30us/sample - loss: 0.6274 - acc: 0.7927
Epoch 4/100
328/328 [==============================] - 0s 40us/sample - loss: 0.4817 - acc: 0.8476
Epoch 5/100
328/328 [==============================] - 0s 27us/sample - loss: 0.3863 - acc: 0.8841
Epoch 6/100
328/328 [==============================] - 0s 33us/sample - loss: 0.3188 - acc: 0.9177
Epoch 7/100
328/328 [==============================] - 0s 33us/sample - loss: 0.2654 - acc: 0.9238
Epoch 8/100
328/328 [==============================] - 0s 27us/sample - loss: 0.2255 - acc: 0.9268
Epoch 9/100
328/328 [==============================] - 0s 33us/sample - loss: 0.1995 - acc: 0.9329
Epoch 10/100
328/328 [==============================] - 0s 30us/sample - loss: 0.1904 -

C:\Users\Administrator\anaconda3\lib\site-packages\keras\engine\training_v1.py:2079: UserWarning: `Model.state_updates` will be removed in a future version. This property should not be used in TensorFlow 2.0, as `updates` are applied automatically.
  updates=self.state_updates,


Epoch 15/100
328/328 [==============================] - 0s 27us/sample - loss: 8.6008e-04 - acc: 1.0000
Epoch 16/100
328/328 [==============================] - 0s 33us/sample - loss: 8.1964e-04 - acc: 1.0000
Epoch 17/100
328/328 [==============================] - 0s 30us/sample - loss: 7.5541e-04 - acc: 1.0000
Epoch 18/100
328/328 [==============================] - 0s 30us/sample - loss: 7.3981e-04 - acc: 1.0000
Epoch 19/100
328/328 [==============================] - 0s 27us/sample - loss: 7.2602e-04 - acc: 1.0000
Epoch 20/100
328/328 [==============================] - 0s 33us/sample - loss: 6.7299e-04 - acc: 1.0000
Epoch 21/100
328/328 [==============================] - 0s 30us/sample - loss: 6.8266e-04 - acc: 1.0000
Epoch 22/100
328/328 [==============================] - 0s 30us/sample - loss: 6.3684e-04 - acc: 1.0000
Epoch 23/100
328/328 [==============================] - 0s 30us/sample - loss: 6.1267e-04 - acc: 1.0000
Epoch 24/100
328/328 [==============================] - 0s 30us/

In [16]:
def get_oof(clf, X_train, y_train, X_test):
    oof_train = np.zeros((X_train.shape[0], _N_CLASS))  
    oof_test = np.empty((X_test.shape[0], _N_CLASS))  

    for i, (train_index, test_index) in enumerate(kf.split(X_train,y_train)):
        kf_X_train = X_train[train_index]  
        kf_y_train = y_train[train_index]   
        kf_X_test = X_train[test_index] 

        clf.fit(kf_X_train, kf_y_train) 

        oof_train[test_index] = clf.predict_proba(kf_X_test) 
        oof_test += clf.predict_proba(X_test)  

    oof_test /= _N_FOLDS 
    return oof_train, oof_test  

In [17]:
oof_train2, oof_test2 = get_oof(dbn, x_train, y_train, x_test)
train2 = Predict(oof_train2)
test2 = Predict(oof_test2)

[START] Pre-training step:
Instructions for updating:
Use `tf.cast` instead.
>> Epoch 1 finished 	RBM Reconstruction error 7.700933
>> Epoch 2 finished 	RBM Reconstruction error 7.501830
>> Epoch 3 finished 	RBM Reconstruction error 7.072241
>> Epoch 4 finished 	RBM Reconstruction error 6.618580
>> Epoch 5 finished 	RBM Reconstruction error 5.767955
>> Epoch 6 finished 	RBM Reconstruction error 5.088319
>> Epoch 7 finished 	RBM Reconstruction error 4.540427
>> Epoch 8 finished 	RBM Reconstruction error 4.130976
>> Epoch 9 finished 	RBM Reconstruction error 3.833757
>> Epoch 10 finished 	RBM Reconstruction error 3.538880
>> Epoch 11 finished 	RBM Reconstruction error 3.354262
>> Epoch 12 finished 	RBM Reconstruction error 3.114407
>> Epoch 13 finished 	RBM Reconstruction error 2.942342
>> Epoch 14 finished 	RBM Reconstruction error 2.803423
>> Epoch 15 finished 	RBM Reconstruction error 2.706394
>> Epoch 16 finished 	RBM Reconstruction error 2.609090
>> Epoch 17 finished 	RBM Reconstruc

In [18]:
oof_train3 = np.zeros((x_train.shape[0], _N_CLASS))  
oof_test3 = np.empty((x_test.shape[0], _N_CLASS)) 

for i, (train_index, test_index) in enumerate(kf.split(x_train,y_train)):
    kf_X_train = x_train[train_index]  
    kf_X_train = kf_X_train.reshape(kf_X_train.shape[0],1,kf_X_train.shape[1])
    kf_y_train = y_train[train_index] 
    kf_y_train = lb.transform(kf_y_train)
    kf_X_test = x_train[test_index]  
    kf_X_test = kf_X_test.reshape(kf_X_test.shape[0],1,kf_X_test.shape[1])

    lstm.fit(kf_X_train, kf_y_train,epochs=1000,verbose=0,batch_size=20)  

    oof_train3[test_index] = lstm.predict(kf_X_test)  
    oof_test3 += lstm.predict(x_test_lstm) 

oof_test3 /= _N_FOLDS  

C:\Users\Administrator\anaconda3\lib\site-packages\keras\engine\training_v1.py:2079: UserWarning: `Model.state_updates` will be removed in a future version. This property should not be used in TensorFlow 2.0, as `updates` are applied automatically.
  updates=self.state_updates,


In [19]:
new_train = []
new_train.append(oof_train1)
new_train.append(oof_train2)
new_train.append(oof_train3)
len(new_train)

3

In [20]:
new_test = []
new_test.append(oof_test1)
new_test.append(oof_test2)
new_test.append(oof_test3)
len(new_test)

3

In [21]:
new_train = np.concatenate(new_train, axis=1)
new_test = np.concatenate(new_test, axis=1)

In [156]:
clf = RandomForestClassifier(n_estimators = 13, criterion="gini",random_state=2)
# clf = RandomForestRegressor()
# clf = MLPClassifier(hidden_layer_sizes=(100, 100), random_state=1000, max_iter=1000)
clf.fit(new_train, y_train_labels)
RFR_pred_train = clf.predict(new_train)
RFR_pred_train_label = Predict(RFR_pred_train)
stock_train_acc = accuracy_score(y_train, RFR_pred_train_label) 
new_test1 = np.nan_to_num(new_test.astype(np.float32))
RFR1_pred = clf.predict(new_test1)
RFR1_pred_label = Predict(RFR1_pred)
stock_test_acc = accuracy_score(y_test, RFR1_pred_label) 
print("stacking training ones score: {}".format(stock_train_acc))
print("stacking testing ones score: {}".format(stock_test_acc))

stacking training ones score: 0.997716894977169
stacking testing ones score: 0.9627659574468085


In [157]:
PUSE_stacking = []
PUSE_stacking.append(y_train)
PUSE_stacking.append(RFR_pred_train_label)
PUSE_stacking.append(y_test)
PUSE_stacking.append(RFR1_pred_label)
test = pd.DataFrame(data=PUSE_stacking,index = ['y_train','ytrain_pred','y_test','ytest_pred'])
test.to_csv(r'./dataset/test/PUSE_stackinginging_combine_multi.csv',encoding='gbk')

<font color=#0099ff  size=5 face="黑体">方法2：stacking 堆叠</font>

In [141]:
oof_train1_1 = np.zeros((new_train.shape[0], _N_CLASS))  
oof_test1_1 = np.empty((new_test1.shape[0], _N_CLASS))  

for i, (train_index, test_index) in enumerate(kf.split(new_train, y_train)):
    kf_X_train = x_train[train_index] 
    kf_y_train = y_train[train_index]  
    kf_y_train = lb.transform(kf_y_train)
    kf_X_test = x_train[test_index]  

    dnn.fit(kf_X_train, kf_y_train,batch_size=20, epochs=100, shuffle=True, verbose=1)  
    
    oof_train1_1[test_index] = dnn.predict(kf_X_test) 
    oof_test1_1 += dnn.predict(new_test1) 

oof_test1_1 /= _N_FOLDS  

Train on 328 samples
Epoch 1/100
328/328 [==============================] - 0s 24us/sample - loss: 1.2081e-06 - acc: 1.0000
Epoch 2/100
328/328 [==============================] - 0s 72us/sample - loss: 1.1939e-06 - acc: 1.0000
Epoch 3/100
328/328 [==============================] - 0s 36us/sample - loss: 1.1841e-06 - acc: 1.0000
Epoch 4/100
328/328 [==============================] - 0s 46us/sample - loss: 1.1841e-06 - acc: 1.0000
Epoch 5/100
328/328 [==============================] - 0s 33us/sample - loss: 1.1641e-06 - acc: 1.0000
Epoch 6/100
328/328 [==============================] - 0s 46us/sample - loss: 1.1536e-06 - acc: 1.0000
Epoch 7/100
328/328 [==============================] - 0s 36us/sample - loss: 1.1427e-06 - acc: 1.0000
Epoch 8/100
328/328 [==============================] - 0s 40us/sample - loss: 1.1358e-06 - acc: 1.0000
Epoch 9/100
328/328 [==============================] - 0s 43us/sample - loss: 1.1274e-06 - acc: 1.0000
Epoch 10/100
328/328 [==============================

ValueError: Error when checking input: expected dense_5_input to have shape (90,) but got array with shape (9,)

In [142]:
oof_train2_2, oof_test2_2 = get_oof(dbn, new_train, y_train, new_test1)

[START] Pre-training step:
>> Epoch 1 finished 	RBM Reconstruction error 1.348823
>> Epoch 2 finished 	RBM Reconstruction error 0.913053
>> Epoch 3 finished 	RBM Reconstruction error 0.551523
>> Epoch 4 finished 	RBM Reconstruction error 0.347839
>> Epoch 5 finished 	RBM Reconstruction error 0.240535
>> Epoch 6 finished 	RBM Reconstruction error 0.176365
>> Epoch 7 finished 	RBM Reconstruction error 0.125811
>> Epoch 8 finished 	RBM Reconstruction error 0.091884
>> Epoch 9 finished 	RBM Reconstruction error 0.078211
>> Epoch 10 finished 	RBM Reconstruction error 0.062592
>> Epoch 11 finished 	RBM Reconstruction error 0.065050
>> Epoch 12 finished 	RBM Reconstruction error 0.055780
>> Epoch 13 finished 	RBM Reconstruction error 0.056126
>> Epoch 14 finished 	RBM Reconstruction error 0.051812
>> Epoch 15 finished 	RBM Reconstruction error 0.056613
>> Epoch 16 finished 	RBM Reconstruction error 0.052684
>> Epoch 17 finished 	RBM Reconstruction error 0.053891
>> Epoch 18 finished 	RBM Reco

In [143]:
new_test_lstm = new_test1.reshape(new_test1.shape[0],1,new_test1.shape[1])
new_test_lstm.shape

(188, 1, 9)

In [144]:
lstm_1 = buildLSTM(timeStep=1,inputColNum=9,outStep=3,learnRate=1e-4)
oof_train3_3 = np.zeros((new_train.shape[0], _N_CLASS))  
oof_test3_3 = np.empty((new_test1.shape[0], _N_CLASS))  

for i, (train_index, test_index) in enumerate(kf.split(new_train, y_train)):
    kf_X_train = new_train[train_index]  
    kf_X_train = kf_X_train.reshape(kf_X_train.shape[0],1,kf_X_train.shape[1])
    kf_y_train = y_train[train_index]  
    kf_y_train = lb.transform(kf_y_train)
    kf_X_test = new_train[test_index] 
    kf_X_test = kf_X_test.reshape(kf_X_test.shape[0],1,kf_X_test.shape[1])

    lstm_1.fit(kf_X_train, kf_y_train,epochs=1000,verbose=0,batch_size=20)  

    oof_train3_3[test_index] = lstm_1.predict(kf_X_test)  
    oof_test3_3 += lstm_1.predict(new_test_lstm) 

oof_test3_3 /= _N_FOLDS

Model: "model_5"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 input_6 (InputLayer)        [(None, 1, 9)]            0         
                                                                 
 lstm_5 (LSTM)               (None, 100)               44000     
                                                                 
 dense_16 (Dense)            (None, 100)               10100     
                                                                 
 dense_17 (Dense)            (None, 3)                 303       
                                                                 
Total params: 54,403
Trainable params: 54,403
Non-trainable params: 0
_________________________________________________________________


C:\Users\Administrator\anaconda3\lib\site-packages\keras\engine\training_v1.py:2079: UserWarning: `Model.state_updates` will be removed in a future version. This property should not be used in TensorFlow 2.0, as `updates` are applied automatically.
  updates=self.state_updates,


In [145]:
new_train_1 = []
new_train_1.append(oof_train1_1)
new_train_1.append(oof_train2_2)
new_train_1.append(oof_train3_3)

In [146]:
new_test_1 = []
new_test_1.append(oof_test1_1)
new_test_1.append(oof_test2_2)
new_test_1.append(oof_test3_3)

In [147]:
new_train_1 = np.concatenate(new_train_1, axis=1)  
new_test_1 = np.concatenate(new_test_1, axis=1)   

In [148]:
clf1 = RandomForestRegressor()
clf1.fit(new_train_1, y_train_labels)

RandomForestClassifier(n_estimators=33, random_state=4)

In [149]:
RFR_pred_train_1 = clf1.predict(new_train_1)
RFR_pred_train_label_1 = Predict(RFR_pred_train_1)
accuracy_score(y_train, RFR_pred_train_label_1) 

1.0

In [170]:
new_test_11 = np.nan_to_num(new_test_1.astype(np.float32))
RFR_pred_1 = clf1.predict(new_test_11)
RFR_pred_label_1 = Predict(RFR_pred_1)
accuracy_score(y_test, RFR_pred_label_1) 

0.6276595744680851